In [1]:
#Imports & db connection setup
from modules.preamble import *
from modules.event_data_conversion import get_time_between_events, events_to_rt_pred

#Enter information to access your PostgreSQL MIMIC database from Python
con = psycopg2.connect(dbname='mimic', user='postgres', password='postgres', port=5433) #Your credentials for the database connector
query_schema = 'set search_path to mimiciii;' #Your search path to the MIMIC-III schema

%matplotlib inline

C:\Users\s113277\.conda\envs\base2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\s113277\.conda\envs\base2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\s113277\.conda\envs\base2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\s113277\.conda\envs\base2\lib\site-packages\tensorflow\python\frame

# Loading data
Including some preliminary cleaning & merging.

In [2]:
#Load datasets from the "Event Tables Filtering" notebook
df_bm = pd.read_hdf(os.path.join(data_base_path, "cleaned_BM_events.hdf"));
df_lt = pd.read_hdf(os.path.join(data_base_path, "cleaned_LT_events.hdf"));

In [3]:
#Drop the at3 variable because it was only measured 4 times in total
df_lt = df_lt[~(df_lt['variable']=='at3')]

In [4]:
#Drop all events with valuenum = NaN. They cannot be used anyway
df_bm = df_bm[~df_bm['valuenum'].isnull()]
df_lt = df_lt[~df_lt['valuenum'].isnull()]

In [5]:
#Concatenated into a single dataframe
df = pd.concat([df_bm, df_lt]).reset_index(drop=True)

# Events-to-modeling format
Use the full forward imputation per patient, as we found this to be the most effective imputation strategy.

In [6]:
#First dataset for modeling
df_mod1 = events_to_rt_pred(df = df[['subject_id', 'charttime', 'variable', 'valuenum']],
                            max_cft = pd.Timedelta('200y'),
                            verbose=True)

Filling variable 0: arterial base excess
Filling variable 1: arterial pco2
Filling variable 2: arterial ph
Filling variable 3: arterial po2
Filling variable 4: bicarbonate
Filling variable 5: bilirubin
Filling variable 6: calcium
Filling variable 7: creatinine
Filling variable 8: crp
Filling variable 9: cvp
Filling variable 10: diastolic blood pressure
Filling variable 11: fio2
Filling variable 12: glucose
Filling variable 13: got(asat)
Filling variable 14: gpt(alat)
Filling variable 15: heart rate
Filling variable 16: hematocrit
Filling variable 17: platelets
Filling variable 18: potassium
Filling variable 19: ptt
Filling variable 20: sodium
Filling variable 21: spo2
Filling variable 22: systolic blood pressure
Filling variable 23: temperature
Filling variable 24: urea
Filling variable 25: white blood cells


Add the target label again:

In [7]:
def extract_TOD(database_connection):
    """
    Extracts table with the times of death of the septic shock patients. 
    Returns the result as a dataframe.
    
    inputs
    -----
    database_connection: database connection to the MIMIC-III PostgreSQL database.
    """
    
    qry = query_schema + """ 
        WITH ss_patients AS (
            SELECT *
            FROM explicit_sepsis
            WHERE septic_shock=1
        )
        
        SELECT
            a.subject_id, max(a.deathtime) as deathtime
        FROM
            admissions AS a 
            INNER JOIN
                ss_patients AS ssp 
                    ON 
                        a.subject_id = ssp.subject_id 
                        AND
                        a.hadm_id = ssp.hadm_id
        GROUP BY a.subject_id
        ORDER BY a.subject_id
        
    """
    df = pd.read_sql(qry, database_connection, parse_dates = 'deathtime')
    return df

In [8]:
df_tod = extract_TOD(con)

df_mod1 = df_mod1.merge(df_tod, on='subject_id', how='left')
df_mod1['time to death'] = df_mod1['deathtime'] - df_mod1['charttime']

    #Add the label we are interested in: mortality within 72h
df_mod1['y'] = (df_mod1['time to death'] <= pd.Timedelta('72 hours')).astype(int).values

## Patient/case based split

In [9]:
#Randomly put split train-val-test 70-%-15%-15% based on patients
from sklearn.model_selection import train_test_split

random_state= 10 #Trial & error with random state such that we approximately have 70%-15%-15%

#Select all subjects and whether they have y=1 eventually
all_subjects = df_mod1['subject_id'].unique()
all_subj_labels = df_mod1.groupby('subject_id')['y'].max().sort_index().values

#Make a split among subjects, stratified on the target label
#First set 30% aside, and then split that 30% 50-50 to get the desired 70%-15%-15%
train_subj, test_subj, train_subj_labels, test_subj_labels = train_test_split(
    all_subjects, 
    all_subj_labels,
    test_size=0.3,
    random_state=random_state,
    stratify=all_subj_labels)

val_subj, test_subj = train_test_split(
    test_subj,
    test_size=0.5,
    random_state=random_state,
    stratify=test_subj_labels)

In [10]:
#Split the instances for modeling based on the subjects
df_train, df_val, df_test = [df_mod1[df_mod1['subject_id'].isin(train_subj)],
                             df_mod1[df_mod1['subject_id'].isin(val_subj)], 
                             df_mod1[df_mod1['subject_id'].isin(test_subj)]]

#Check if split conforms to requirements
print("Patients in train, val test sets:", len(train_subj), len(val_subj), len(test_subj))
print("Observations in train, val test sets:", len(df_train), len(df_val), len(df_test))
print("% of total observations in train, val, test sets",
      len(df_train)/len(df_mod1)*100,
      len(df_val)/len(df_mod1)*100,
      len(df_test)/len(df_mod1)*100, "\n")
print('Fraction y=1 in train, val, test sets:', df_train['y'].mean(), df_val['y'].mean(), df_test['y'].mean())

Patients in train, val test sets: 1691 362 363
Observations in train, val test sets: 688457 117419 118967
% of total observations in train, val, test sets 74.44041853590285 12.696100851712128 12.86348061238502 

Fraction y=1 in train, val, test sets: 0.0991783074324177 0.10306679498207275 0.09909470693553675


In [11]:
#Re-sort all datasets based on instances & time
df_train = df_train.sort_values(['subject_id', 'charttime']).reset_index(drop=True)
df_val = df_val.sort_values(['subject_id', 'charttime']).reset_index(drop=True)
df_test = df_test.sort_values(['subject_id', 'charttime']).reset_index(drop=True)

## Imputation & normalization

In [12]:
#Storing train-val-test features & labels in numpy format
feat_colnames = df_train.columns[2:-3]
X_train, X_val, X_test = df_train[feat_colnames].values, \
    df_val[feat_colnames].values, \
    df_test[feat_colnames].values
y_train, y_val, y_test = df_train['y'].values, df_val['y'].values, df_test['y'].values

#Checking shapes
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((688457, 26), (117419, 26), (118967, 26), (688457,), (117419,), (118967,))

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#Imputation
si = SimpleImputer(strategy='mean')
X_train = si.fit_transform(X_train)
X_val = si.transform(X_val)
X_test = si.transform(X_test)

#Scaling / normalization
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

In [14]:
#Store means & stds used for imputation to recover original scale later
means = ss.mean_
stds = np.sqrt(ss.var_)

In [15]:
#Brief check - is the mean now 0 and std now 1?
pd.DataFrame(X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
count,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05,6.884570e+05
mean,2.116436e-13,4.160303e-14,-1.055183e-14,3.791913e-14,1.370912e-13,1.698510e-14,3.876798e-14,6.460419e-14,6.156123e-14,2.066529e-15,4.050591e-15,-4.985240e-14,-3.061817e-14,-7.972327e-15,1.499832e-15,1.589692e-14,-1.646681e-15,3.944007e-14,-1.768761e-14,-1.421181e-14,4.660061e-14,-5.213436e-13,-1.205046e-14,-2.197143e-14,-3.190386e-14,4.919475e-14
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-8.429132e+00,-2.528253e+00,-1.311051e+01,-1.987826e+00,-3.027848e+00,-4.587794e-01,-7.603983e+00,-1.213542e+00,-4.708129e+00,-3.578643e+00,-2.988981e+00,-3.075588e+00,-2.281377e+00,-1.906584e-01,-2.757858e-01,-4.923800e+00,-6.231042e+00,-1.396901e+00,-4.507422e+00,-1.212660e+00,-7.298593e+00,-1.561772e+01,-4.557371e+00,-7.979223e+00,-1.266837e+00,-1.427581e+00
25%,-6.655641e-01,-5.814442e-01,-5.758826e-01,-5.532557e-01,-6.628083e-01,-3.835310e-01,-6.402645e-01,-6.705058e-01,0.000000e+00,5.573284e-16,-6.704212e-01,-6.315674e-01,-5.763983e-01,-1.615546e-01,-2.206083e-01,-7.222200e-01,-6.744245e-01,-7.776170e-01,-6.856271e-01,-5.680196e-01,-6.669183e-01,-3.647201e-01,-6.957136e-01,-5.647793e-01,-6.792776e-01,-6.114259e-01
50%,9.528693e-03,-9.474196e-02,6.363918e-02,-1.073758e-01,-3.213100e-02,-3.270947e-01,-5.011879e-02,-3.688190e-01,0.000000e+00,5.573284e-16,-9.078128e-02,-2.056221e-02,-1.762504e-01,-1.373014e-01,-1.826737e-01,-3.082076e-02,-9.074618e-02,-1.833547e-01,-1.871320e-01,-3.182942e-01,-8.177058e-02,1.984675e-01,-1.056716e-01,-2.554492e-02,-3.155505e-01,-2.089383e-01
75%,5.158483e-01,3.108433e-01,7.031610e-01,3.385041e-01,5.985463e-01,-6.372540e-02,5.400269e-01,4.155666e-01,0.000000e+00,5.573284e-16,5.661440e-01,4.341453e-16,3.456817e-01,-7.424316e-02,-6.542138e-02,6.924892e-01,5.162793e-01,5.547818e-01,6.436931e-01,1.485450e-01,6.984264e-01,5.739258e-01,6.074958e-01,5.136853e-01,3.279667e-01,3.947932e-01
max,5.241498e+00,9.233718e+00,4.796101e+00,1.022541e+01,4.067271e+00,1.500476e+01,2.261148e+01,1.206068e+01,4.171971e+00,7.088789e+00,7.058111e+00,3.034464e+00,5.967196e+01,3.833469e+01,2.870967e+01,6.510879e+00,7.520419e+00,7.726590e+00,1.152750e+01,5.760443e+00,8.305347e+00,8.085873e-01,4.077391e+00,6.849665e+00,4.636734e+00,2.159248e+01


In [16]:
#Did we use the mean of the training set for the test as well, to avoid leakage?
pd.DataFrame(X_train).mode()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,0.009529,-1.152743e-15,0.0,2.754928e-16,-0.1898,-0.421155,0.06791,-0.610168,0.0,5.573284e-16,-0.206709,-0.020562,-0.141455,4.595443e-17,-0.21716,-0.669035,-0.417606,1.649476,-0.353297,-0.337951,-0.27682,0.808587,2.045122e-15,-0.227759,-0.483425,-0.644967


In [17]:
#Did we use the mean of the training set for the test as well, to avoid leakage?
pd.DataFrame(X_test).mode()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,0.009529,-1.152743e-15,0.0,2.754928e-16,-0.1898,-0.402343,-0.168148,-0.79118,0.0,5.573284e-16,0.102432,-0.020562,-0.124057,4.595443e-17,-4.900752e-17,-0.13719,0.749751,-0.471103,-0.187132,3.491676e-16,-0.471869,0.808587,2.045122e-15,-0.227759,-0.735236,-0.343101


#  Instance weights

In [18]:
#Compute standalone sample weights (without class weights)
instance_weights = []

for dfx in [df_train, df_val, df_test]:
    #Dataframe with sample weights, initially contains counts (number of instances) per patient
    dfx_weights = dfx\
        .groupby('subject_id')['charttime']\
        .count()\
        .to_frame()\
        .reset_index()\
        .rename(columns={'charttime':'instance_weight'}) #Be sure to pick column without missing values

    #Compute weights from the counts based on the number of instances per patient
    dfx_weights['instance_weight'] = dfx_weights['instance_weight'].apply(lambda x: 1/x)

    #Converting sample weights to the shape that fits X, i.e. row-per-instance
    instance_weight = dfx.merge(dfx_weights, on='subject_id', how='left')['instance_weight'].values
    
    #Store result for use outside this loop
    instance_weights.append(instance_weight)
    
instance_weights_train, instance_weights_val, instance_weights_test = instance_weights

# Export data to pickles for easy re-use in other notebooks

In [32]:
#Dataframes
# for dfx, name in zip([df_train, df_val, df_test], ['df_train', 'df_val', 'df_test']):
#     dfx.to_hdf(path_or_buf=os.path.join(data_base_path, "modeling_data/" + name + ".hdf"),
#                key='Processed_data',
#                mode='w',
#                complevel=9
#             )

In [33]:
#Processed features & target labels
# for X_, y_, X_name, y_name in zip([X_train, X_val, X_test], [y_train, y_val, y_test], ['X_train', 'X_val', 'X_test'], ['y_train', 'y_val', 'y_test']):
#     pd.DataFrame(X_).to_hdf(
#         path_or_buf=os.path.join(data_base_path, "modeling_data/" + X_name + ".hdf"),
#         key='Processed_data',
#         mode='w',
#         complevel=9
#     )
    
#     pd.DataFrame(y_).to_hdf(
#         path_or_buf=os.path.join(data_base_path, "modeling_data/" + y_name + ".hdf"),
#         key='Processed_data',
#         mode='w',
#         complevel=9
#     )

In [34]:
#Means & stds for recovering original scale
# pd.DataFrame(means).to_hdf(
#     path_or_buf=os.path.join(data_base_path, "modeling_data/" + 'means' + ".hdf"),
#     key='Processed_data',
#     mode='w',
#     complevel=9
# )

# pd.DataFrame(stds).to_hdf(
#     path_or_buf=os.path.join(data_base_path, "modeling_data/" + 'stds' + ".hdf"),
#     key='Processed_data',
#     mode='w',
#     complevel=9
# )

In [35]:
#Instance weights
# for instance_weights, name in zip([instance_weights_train, instance_weights_val, instance_weights_test], ['instance_weights_train', 'instance_weights_val', 'instance_weights_test']):
#     pd.DataFrame(instance_weights).to_hdf(
#         path_or_buf=os.path.join(data_base_path, "modeling_data/" + name + ".hdf"),
#         key='Processed_data',
#         mode='w',
#         complevel=9
#     )